**# 多输入多输出通道**

**1. 多输入**

* 实现多输入通道的互相关运算

In [ ]:
!pip install d2l
!pip install matplotlib==3.0

In [40]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [41]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

**2. 多输出**

* 计算多个通道的输出的互相关函数

In [42]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [43]:
# stack里0的意思是，在第0维将K，K+1，K+2堆叠形成新的矩阵 
K = torch.stack((K, K + 1, K + 2), 0)
K

tensor([[[[0., 1.],
          [2., 3.]],

         [[1., 2.],
          [3., 4.]]],


        [[[1., 2.],
          [3., 4.]],

         [[2., 3.],
          [4., 5.]]],


        [[[2., 3.],
          [4., 5.]],

         [[3., 4.],
          [5., 6.]]]])

In [44]:
K.shape

torch.Size([3, 2, 2, 2])

In [45]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

**3. 1x1卷积层**

In [46]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [47]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

In [48]:
Y1

tensor([[[ 0.8816,  1.3459,  2.3521],
         [ 1.2766, -0.2651,  3.8822],
         [ 2.7430, -3.4280, -1.7445]],

        [[-0.4806,  0.9816, -0.7432],
         [-0.1802,  2.0091, -0.5267],
         [ 0.4300, -0.1122,  0.2381]]])

In [49]:
Y2

tensor([[[ 0.8816,  1.3459,  2.3521],
         [ 1.2766, -0.2651,  3.8822],
         [ 2.7430, -3.4280, -1.7445]],

        [[-0.4806,  0.9816, -0.7432],
         [-0.1802,  2.0091, -0.5267],
         [ 0.4300, -0.1122,  0.2381]]])